# Lab. 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Data preparation

In [ ]:
functions = [           # 1
    lambda x: 5 * x**3 - 2 * x**2 + 3 * x - 17,
    lambda x: np.sin(x) + np.cos(x),
    lambda x: 2 * np.log(x + 1),
    lambda x, y: x + 2 * y,
    lambda x, y: np.sin(x / 2) + 2 * np.cos(x),
    lambda x, y: x**2 + 3 * x * y - 7 * y + 1,
] + (
    new_functions := [  # 2
        lambda x: np.sin(x + 3.141592 / 2),
        lambda x: np.tan(2 * x + 1),
    ]
)

In [ ]:
asymptote = lambda n: np.round(-np.pi / 4 - 1 / 2 + np.pi * n / 2, 2) # 2

domains = [             # 1
    [(-10, 10), (0, 100), (-1, 1), (-1000, 1000)],
    [(-3.14, 3.14), (0, 7), (0, 100), (-100, 100)],
    [(0, 4), (0, 9), (0, 99), (0, 999)],
    [(0, 1), (-10, 10), (0, 100), (-1000, 1000)],
    [(-3.14, 3.14), (0, 7), (0, 100), (-100, 100)],
    [(-10, 10), (0, 100), (-1, 1), (-1000, 1000)],
] + (
    new_domains := [    # 2
        [(0, 6.28), (-3.14, 3.14), (-1.57, 4.71), (-1000, 1000)],
        [(asymptote(0.01), asymptote(0.99)), (-3.14, 3.14), (-1.57, 1.57)],
    ]
)

In [ ]:
import os
import subprocess

In [ ]:
for i, (function, domains_) in enumerate(zip(new_functions, new_domains)):
    for domain in domains_:
        X = np.linspace(*domain, (n if (n := int(100 * abs(domain[1] - domain[0]))) < 1001 else 1000)).round(2)
        Y = function(X).round(2)
        filename = f"1_{i+7}__{domain[0]}_{domain[1]}.dat"
        with open(f"data\input\{filename}", "a") as f:
            f.write(f"1 100 -5 6 {X.shape[0]}\n")
        pd.DataFrame({"X": X, "Y": Y}).to_csv(
            f"data\input\{filename}", index=False, header=False, sep=" ", mode="a"
        )

In [ ]:
input_filenames = os.listdir("data\input")

In [ ]:
def get_domain_from_filename(filename):
    return tuple(map(float, ".".join(filename.split('.')[:-1]).split('__')[1].split('_')))

def get_domain_length_from_filename(filename):
    domain = get_domain_from_filename(filename)
    return abs(domain[1] - domain[0])

def get_problem_solving_threshold_from_filename(filename):
    domain_length = get_domain_length_from_filename(filename)
    return domain_length * 0.21

In [ ]:
for input_file in input_filenames:
    output_file = ".".join(input_file.split('.')[:-1]) + '.out'
    problem_solved_threshold = get_problem_solving_threshold_from_filename(input_file)
    print(
        command := f"copy NUL data\output\with_sin_cos\{output_file} & cd TinyGP & java TinyGP.java ..\data\input\{input_file} {problem_solved_threshold} > ..\data\output\with_sin_cos\{output_file} & cd .."
    )
    t = subprocess.run(command, shell=True)

### Output parsing

In [ ]:
import re
import json

In [ ]:
from numpy import sin, cos

def dictify(log):
    return (
        log.replace("AvgFitness", '{"avg_fitness"')
        .replace("BestFitness", ', "best_fitness"')
        .replace("AvgSize", ', "avg_size"')
        .replace("BestIndividual:", ', "best_individual": "')
        .replace("\n", " ")
        .replace("=", ": ")
        .replace("X1", "X")
        .replace("X2", "Y")
        .replace("SIN", "sin")
        .replace("COS", "cos")
        .replace("PROBLEM*NOT*SOLVED", "")
        .replace("PROBLEMSOLVED", "")
        + '"}'
    )

In [ ]:
def logs_to_df(logs):
    generations = {
        int(log.split(" ")[0][1:]): "".join(log.split(" ")[1:])
        for log in re.split("Generation", logs)[1:]
    }
    for k, v in generations.items():
        generations[k] = json.loads(dictify(v))
    return pd.DataFrame().from_dict(generations, orient="index")

### Simplification

In [ ]:
import sympy as sp
import inspect

In [ ]:
def simplify(expression):
    return str(sp.simplify(expression))

### Comparison

In [ ]:
def compare(function, domain, df, size=(8, 8)):
    X = np.linspace(*domain, int(100 * abs(domain[1] - domain[0])))
    Y_true = function(X)

    best_individual = df.iloc[-1, -1]
    Y_gp = eval(best_individual)

    plt.figure(figsize=size)
    plt.plot(X, Y_true, label="True", linestyle="-", color="black", linewidth=4, alpha=0.5)
    plt.plot(X, Y_gp, label="GP", linestyle="-", color="red", linewidth=1)
    plt.legend()
    plt.title(
        inspect.getsource(function)
        .replace("lambda x:", "f(x) =")
        .replace(",", "")
        .replace("np.", "")
        + f"\nx ∈ [{domain[0]}, {domain[1]}]"

    )
    plt.grid()
    plt.show()
    print(f"Simplified best individual: {simplify(best_individual)}")

In [ ]:
def compare_2_var(function, domain, df, size=(8, 8)):
    X, Y = np.meshgrid(np.linspace(*domain, 100), np.linspace(*domain, 100))
    Z_true = function(X.ravel(), Y.ravel())

    best_individual = df.iloc[-1, -1]
    Z_gp = eval(best_individual)

    plt.figure(figsize=size)
    ax = plt.axes(projection="3d")
    ax.plot_surface(
        X,
        Y,
        Z_true.reshape(X.shape),
        label="True",
        color="gray",
        linewidth=2,
        alpha=0.5,
    )
    ax.plot_surface(
        X, 
        Y, 
        Z_gp.reshape(X.shape), 
        label="GP", 
        color="red", 
        linewidth=1, 
        alpha=0.5
    )
    plt.legend()
    plt.title(
        inspect.getsource(function)
        .replace(",", "")
        .replace("lambda x y:", "f(x, y) =")
        .replace("np.", "")
        + f"\nx ∈ [{domain[0]}, {domain[1]}], y ∈ [{domain[0]}, {domain[1]}]"
    )
    plt.grid()
    plt.show()
    print(f"Simplified best individual: {simplify(best_individual)}")

In [ ]:
def compare_2_var_3_views(function, domain, df, size=(12, 5)):
    X, Y = np.meshgrid(np.linspace(*domain, 100), np.linspace(*domain, 100))
    Z_true = function(X.ravel(), Y.ravel())

    best_individual = df.iloc[-1, -1]
    Z_gp = eval(best_individual)

    fig = plt.figure(figsize=size)
    fig.suptitle(
        inspect.getsource(function)
        .replace(",", "")
        .replace("lambda x y:", "f(x, y) =")
        .replace("np.", "")
        + f"\nx ∈ [{domain[0]}, {domain[1]}], y ∈ [{domain[0]}, {domain[1]}]"
    )

    ax = fig.add_subplot(1, 3, 1, projection='3d')

    surf1 = ax.plot_surface(X, Y, Z_true.reshape(X.shape), rstride=1, cstride=1,
                        linewidth=0, antialiased=False, color="gray", alpha=0.5, label="True")

    surf2 = ax.plot_surface(X, Y, Z_gp.reshape(X.shape), rstride=1, cstride=1,
                    linewidth=0, antialiased=False, color="red", alpha=0.5, label="GP")
    ax.xaxis.set_label_text("X")
    ax.yaxis.set_label_text("Y")
    ax.zaxis.set_label_text("Z")


    ax = fig.add_subplot(1, 3, 2, projection='3d')
    surf1 = ax.plot_surface(X, Y, Z_true.reshape(X.shape), rstride=1, cstride=1,
                    linewidth=0, antialiased=False, color="gray", alpha=0.5, label="True")

    surf2 = ax.plot_surface(X, Y, Z_gp.reshape(X.shape), rstride=1, cstride=1,
                    linewidth=0, antialiased=False, color="red", alpha=0.5, label="GP")
    ax.xaxis.set_label_text("X")
    ax.yaxis.set_label_text("Y")
    ax.zaxis.set_label_text("Z")
    ax.view_init(azim=90) 


    ax = fig.add_subplot(1, 3, 3, projection='3d')
    surf1 = ax.plot_surface(X, Y, Z_true.reshape(X.shape), rstride=1, cstride=1,
                    linewidth=0, antialiased=False, color="gray", alpha=0.5, label="True")

    surf2 = ax.plot_surface(X, Y, Z_gp.reshape(X.shape), rstride=1, cstride=1,
                    linewidth=0, antialiased=False, color="red", alpha=0.5, label="GP")
    ax.xaxis.set_label_text("X")
    ax.yaxis.set_label_text("Y")
    ax.zaxis.set_label_text("Z")
    ax.view_init(azim=180)

    plt.grid()
    plt.legend()
    plt.show()
    print(f"Simplified best individual: {simplify(best_individual)}")


In [ ]:
def show_fitness_process(df):
    f, a = plt.subplots(2)
    a[0].plot(
        df["avg_fitness"], label="avg_fitness", linestyle="-", color="red", linewidth=1
    )
    a[0].legend()
    a[0].grid()
    a[1].plot(
        df["best_fitness"],
        label="best_fitness",
        linestyle="-",
        color="blue",
        linewidth=1,
    )
    a[1].legend()
    a[1].grid()
    a[0].title.set_text("fitness(generation)")
    plt.show()

In [ ]:
for i, (function, domains_) in enumerate(zip(functions, domains)):
    print(
        f'Function {i+1}: {inspect.getsource(function).replace("lambda x:", "f(x) =").replace("lambda x, y:", "f(x, y) =")}'
    )
    for domain in domains_:

        with open(f"data/output/with_sin_cos/1_{i+1}__{domain[0]}_{domain[1]}.out", "r") as f:
            logs = f.read()

        df = logs_to_df(logs)

        if i in [3, 4, 5]:
            compare_2_var_3_views(function, domain, df)
        else:
            compare(function, domain, df, size=(6, 6))

        # show_fitness_process(df)

### BTC course vs. TinyGP 

In [ ]:
btc_df = pd.read_csv("data/other/btc.csv")
btc_df.sort_values(by="Date", inplace=True)
btc_df.reset_index(drop=True, inplace=True)
btc_df.head()

In [ ]:
BTC_FILENAME = "btc.dat"
BTC_DIR = "data/other/btc.dat"

with open(BTC_DIR, "a") as f:
    f.write(f"1 100 -10 10 {btc_df.index.size}\n")

btc_df[["Price"]].apply(lambda x: x.str.replace(",", "")).to_csv(
    BTC_DIR, index=True, header=False, sep=" ", mode="a"
)

In [ ]:
input_file = BTC_FILENAME
output_file = ".".join(input_file.split(".")[:-1]) + ".out"
print(
    command := f"copy NUL data\other\{output_file} & cd TinyGP & java TinyGP.java ..\data\other\{input_file} > ..\data\other\{output_file} & cd .."
)
subprocess.run(command, shell=True)

In [ ]:
with open(f"data/other/{output_file}", "r") as f:
    logs = f.read()

tinygp_btc_df = logs_to_df(logs)

In [ ]:
X = np.arange(btc_df.index.start, btc_df.index.stop, btc_df.index.step)
Y_true = btc_df["Price"].str.replace(",", "").astype(float)
Y_gp = eval(tinygp_btc_df["best_individual"].iloc[-1])

plt.figure(figsize=(12, 6))
plt.plot(X, Y_true, label="True", linestyle="-", color="black", linewidth=4, alpha=0.5)
plt.plot(X, Y_gp, label="GP", linestyle="-", color="red", linewidth=1)
plt.ylabel("Price [USD]")
plt.xlabel("Days (since 23/05/2023)")
plt.legend()
plt.title("BTC price")
plt.grid()
plt.show()